# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [4]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        '''ToDo: fix this function'''
        return 1.0 * sum(self.w) / self.C

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        '''ToDo: fix this function'''
        if (answer * self.f(x) <= 1):
            return -1.0
        return 0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]
            
            # simple heuristic for step size
            step = 0.5 * 0.9 ** k

            # w update
            '''ToDo: add w update with regularization'''
            self.w -= step * x* y * (self.der_loss(x, y) + self.der_reg())
            
            # w_0 update
            '''ToDo: add w_0 update'''
            self.w0 -= step * y * self.der_loss(x, y) 

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [5]:
model = MySVM()
model.fit(X_train, y_train)
print(model.w, model.w0)

[-0.41041213  2.00638367  1.44100532  2.24153224  1.26971889  1.5619352
 -1.86468053 -0.33422479  0.04976797 -2.05723905 -0.92618973 -0.69184034
  1.49998359 -2.44249558 -1.62607581 -0.05544792  0.34755316  4.76603704
  2.31798329  0.31086402] -0.8289694298107237


In [6]:
predictions = model.predict(X_test)

In [7]:
print(predictions)

[0 1 0 ... 1 1 1]


In [8]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [9]:
print(len(predictions), sum(predictions))

2000 1186


In [10]:
print(sum(predictions == y_test) / float(len(y_test)))

0.6085


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [11]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

In [12]:
sklearn_model = LinearSVC(C=10000, loss='hinge', max_iter=10000)
sklearn_model.fit(X_train, y_train)

D:\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(C=10000, loss='hinge', max_iter=10000)

In [13]:
predict = sklearn_model.predict(X_test)

In [14]:
print(accuracy_score(y_test, predict))

0.717
